<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Steam games dataset 2024
</div>

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
<strong>Recuperación de información</strong>

<p>
Recuperdor de información, para saber  que tipo de contenido se desea como resumenes, detallesm reseñas, edades
</p>
</div>

In [1]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

In [4]:
df = pd.read_csv('games_may2024_cleaned.csv')